Checking the GPU config allotted under Google Colab Pro

In [ ]:
!nvidia-smi

Sun Mar 28 17:14:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if not ram_gb < 20:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


Adding simple colab hacks to extend session timeout with few lines of JavaScript code. Inspect element and run this in console.

In [ ]:
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
# }
# setInterval(ConnectButton,60000);

# **Fine-tuning XLSR-Wav2Vec2 for Marathi ASR with 🤗 Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2`s ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

Note that in this notebook, we will fine-tune XLSR-Wav2Vec2 without making use of a language model. We fine-tune the [wav2vec2-large-xlsr-53](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) on the low resource Marathi ASR dataset of [OpenSLR-64](http://openslr.org/64/) that contains just ~2h of validated training data.

XLSR-Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `torchaudio` and `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

## Prepare Data, Tokenizer, Feature Extractor and Create Wav2Vec2CTCTokenizer

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the XLSR-Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).



The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned XLSR-Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on XLSR-Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Common Voice and define a vocabulary based on the dataset's transcriptions.

In [ ]:
import pandas as pd
import numpy as np
import time
import os, sys
from datasets import Dataset

Mount your google drive to read dataset and to save the trained model after completing fine-tuning.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


First, let's go to [Open SLR website](http://openslr.org/64/) and download dataset for marathi language to fine-tune XLSR-Wav2Vec2 on.



Because the Marathi dataset is so small, we will merge both the validation and training data into a training dataset and simply use the test data for validation.

In [ ]:
dataset_path_in_drive = "/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_in_female/"

In [ ]:
audio_df = pd.read_csv(os.path.join(dataset_path_in_drive,'line_index.tsv'),sep='\t',header=None)

In [ ]:
audio_df.shape

(1569, 2)

In [ ]:
audio_df.head()

,0,1
0,mrt_01523_01464306257,एक बस चार तासांपूर्वी येऊन गेली आणि पुढची बस य...
1,mrt_04310_01727087719,अमरापूर गावाला रुपेरी पडद्यावर ओळख निर्माण करू...
2,mrt_03398_00598923081,आज लंडनमध्ये कमाल तापमान पंचवीस अंश सेल्सिअस अ...
3,mrt_02624_01527619254,आता ती छत्री कुठे आहे ?
4,mrt_01523_00829338962,मी तुमच्यासाठी काय करू शकते ?


In [ ]:
audio_df.columns

Int64Index([0, 1], dtype='int64')

In [ ]:
audio_df.columns = ['path_in_folder','actual_text']

In [ ]:
audio_df['path_in_folder'] = audio_df['path_in_folder'].apply(lambda x: dataset_path_in_drive + x + '.wav')

In [ ]:
audio_df = audio_df.sample(frac=1, random_state=2020).reset_index(drop=True)

In [ ]:
audio_df.head(10)

,path_in_folder,actual_text
0,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,"आपण पृथ्वीविषयी का बोलत नाही ? चला , आपण पुढच्..."
1,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,अर्चना ह्या सुट्टीत बोगोताला जाऊन तिथे चांगली ...
2,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,अश्विनीचा जन्म दोन हजार तीन सालातील चैत्र या म...
3,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,अर्चना परवा हवानामधे गेली होती तेव्हा तिथल्या ...
4,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,मी खरंच तरूण आहे का ?
5,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,आपल्या सुरक्षकवचातून बाहेर पडा
6,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,गांगुलीवरच्या एका सामन्याच्या बंदीमुळे मालिकेत...
7,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,तुम्हालाही भेटून आनंद झाला . मी तुम्हाला कशाप्...
8,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,आज कॅलिफोर्नियामध्ये कमाल तापमान सदुसष्ट अंश फ...
9,/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_...,अर्चना परवा डर्बनमधे गेली होती तेव्हा तिथल्या ...


In [ ]:
all_data = Dataset.from_pandas(audio_df)

In [ ]:
all_data

Dataset({
    features: ['path_in_folder', 'actual_text'],
    num_rows: 1569
})

Split into train(90%) and test set(10%) and set seed for reproducibility

In [ ]:
all_data = all_data.train_test_split(test_size=0.10,seed=2020)

In [ ]:
all_data

DatasetDict({
    train: Dataset({
        features: ['path_in_folder', 'actual_text'],
        num_rows: 1412
    })
    test: Dataset({
        features: ['path_in_folder', 'actual_text'],
        num_rows: 157
    })
})

In [ ]:
mr_train_dataset = all_data['train']
mr_test_dataset = all_data['test']

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(mr_train_dataset.remove_columns(["path_in_folder"]), num_examples=50)

,actual_text
0,अर्चना परवा कराचीमधे गेली होती तेव्हा तिथल्या एका रस्त्यावर तिला गॅरी कास्पारोव्हचा पुतळा दिसला
1,एकोणपन्नास दशांश नऊ नऊ डॉलरचे बेस्ट बायचे कॅमेरे गुगलवरून विकत घेता येतील
2,केतकीचा जन्म दोन हजार एक सालातील अश्विन ह्या महिन्यात गडचिरोली इथे झाला
3,मी तुम्हाला काय मदत करू शकते ?
4,अठराशे पंन्नास साली वैद्यकीचा अभ्यासक्रम पुरा करणाऱ्या पदवीधरांच्या पहिल्या तुकडीत ते होते
5,एवढा राग कशामुळे ?
6,अनुजाचा जन्म दोन हजार सोळा सालातील ऑक्टोबर या महिन्यात अकोला इथे झाला
7,अश्विनीचा जन्म दोन हजार एक सालातील कार्तिक या महिन्यात भुसावळ इथे झाला
8,अनुजाचा जन्म एकोणीसशे नव्व्याण्णव सालातील चैत्र ह्या महिन्यात कोरेगांव इथे झाला
9,गायत्री ह्या सुट्टीत हवानाला जाऊन तिथे चांगली आठ दिवस राहून अख्खं शहर पालथं घालणार आहे


The transcriptions look fairly clean.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["actual_text"] = re.sub(chars_to_ignore_regex, '', batch["actual_text"]).lower() + " "
    return batch

In [ ]:
mr_train_dataset = mr_train_dataset.map(remove_special_characters)
mr_test_dataset = mr_test_dataset.map(remove_special_characters)

In [ ]:
show_random_elements(mr_train_dataset.remove_columns(["path_in_folder"]))

,actual_text
0,आता अ‍ॅक्सिस बँकेची सोळावी शाखा फॅशन स्ट्रीटजवळ उघडते आहे परवा तिचा उद्घाटन समारंभ आहे
1,एक बस दहा तासांपूर्वी येऊन गेली आणि पुढची बस यायला अजून फक्त साठ मिनिटं आहेत
2,गायत्री परवा प्रागमध्ये गेली होती तेव्हा तिथल्या एका रस्त्यावर तिला केनी रॉजर्सचा पुतळा दिसला
3,कुसुमाग्रजांच्या मते लेखक हा सुसंस्कृत मनुष्य असतो
4,अश्विनीचा जन्म दोन हजार तेरा सालातील ऑक्टोबर ह्या महिन्यात बुलढाणा इथे झाला
5,आता अ‍ॅक्सिस बँकेची आठवी शाखा महात्मा फुले मंडईजवळ उघडते आहे परवा तिचा उद्घाटन समारंभ आहे
6,कदाचित मला काही मदत करता येईल
7,एक बस बारा तासांपूर्वी येऊन गेली आणि पुढची बस यायला अजून फक्त दोन सेकंदं आहेत
8,मी तुमच्यासाठी शोध घेऊ शकतो तुम्हाला काय जाणून घ्यायचं आहे
9,अर्चना परवा अलेक्झांड्रियामध्ये गेली होती तेव्हा तिथल्या एका रस्त्यावर तिला मायकेल जॅक्सनचा पुतळा दिसला


 We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["actual_text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = mr_train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=mr_train_dataset.column_names)
vocab_test = mr_test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=mr_test_dataset.column_names)

In [ ]:
vocab_train.data

pyarrow.Table
all_text: string
vocab: list<item: string>
  child 0, item: string

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 63,
 'ँ': 29,
 'ं': 18,
 'ः': 3,
 'अ': 5,
 'आ': 39,
 'इ': 40,
 'ई': 8,
 'उ': 59,
 'ऊ': 28,
 'ऋ': 33,
 'ए': 56,
 'ऐ': 4,
 'ऑ': 53,
 'ओ': 23,
 'औ': 38,
 'क': 1,
 'ख': 2,
 'ग': 31,
 'घ': 9,
 'च': 55,
 'छ': 36,
 'ज': 60,
 'झ': 61,
 'ञ': 17,
 'ट': 64,
 'ठ': 45,
 'ड': 34,
 'ढ': 26,
 'ण': 10,
 'त': 54,
 'थ': 6,
 'द': 51,
 'ध': 44,
 'न': 48,
 'प': 35,
 'फ': 52,
 'ब': 7,
 'भ': 43,
 'म': 25,
 'य': 42,
 'र': 15,
 'ऱ': 24,
 'ल': 49,
 'ळ': 27,
 'व': 20,
 'श': 50,
 'ष': 19,
 'स': 16,
 'ह': 57,
 'ा': 11,
 'ि': 12,
 'ी': 0,
 'ु': 62,
 'ू': 46,
 'ृ': 30,
 'ॅ': 21,
 'े': 13,
 'ै': 47,
 'ॉ': 58,
 'ो': 41,
 'ौ': 14,
 '्': 32,
 'ॲ': 65,
 '\u200c': 22,
 '\u200d': 37}

In [ ]:
len(vocab_dict)

66

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted some special characters. Note that we did not exclude all special characters because: 

- The model has to learn to predict when a word is finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- You can also experiment with schwa deletion and retention rules for marathi

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Common Voice's training set. 

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

68

Cool, now our vocabulary is complete and consists of 68 tokens, which means that the linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint will have an output dimension of 68.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create XLSR-Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLSR-Wav2Vec2 was pretrained on the audio data of [Babel](https://huggingface.co/datasets/librispeech_asr), 
[Multilingual LibriSpeech (MLS)](https://ai.facebook.com/blog/a-new-open-data-set-for-multilingual-speech-research/), and [Common Voice](https://huggingface.co/datasets/common_voice). Most of those datasets were sampled at 16kHz, so that Open SLR, sampled at 48kHz, has to be downsampled to 16kHz for training. Therefore, we will have to downsample our fine-tuning data to 16kHz in the following.



A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Great, XLSR-Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of XLSR-Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


We will give the fine-tuned model the name `"wav2vec2-large-xlsr-marathi"`.

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-marathi")

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just kept the path to its file in the dataset. `XLSR-Wav2Vec2` expects the audio file in the format of a 1-dimensional array, so in the first step, let's load all audio files into the dataset object.

Let's first check the serialization format of the downloaded audio files by looking at the first training sample.

In [ ]:
mr_train_dataset[0]

{'actual_text': 'अफगाणिस्तानने सराव सामना सहासष्ट धावांनी जिंकला ',
 'path_in_folder': '/content/gdrive/MyDrive/OpenSLR-64_Marathi/mr_in_female/mrt_04310_01900128283.wav'}

The audio file is saved in the `.wav` format. We found that the [`torchaudio`](https://pytorch.org/audio/stable/index.html) library works best for reading in `.wav` data. 

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a `map(...)` function accordingly.

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path_in_folder"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["actual_text"]
    return batch

In [ ]:
mr_train_dataset = mr_train_dataset.map(speech_file_to_array_fn, remove_columns=mr_train_dataset.column_names)
mr_test_dataset = mr_test_dataset.map(speech_file_to_array_fn, remove_columns=mr_test_dataset.column_names)

We've successfully read in all the audio files, but since we know that Open SLR-64 is sampled at 48kHz, we need to resample the audio files to 16kHz. 

Let's make use of the [`librosa`](https://github.com/librosa/librosa) library to downsample the data.

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
# mr_train_dataset = mr_train_dataset.map(resample, num_proc=4)
# mr_test_dataset = mr_test_dataset.map(resample, num_proc=4)
#not using multiple processes #known bug hf datasets as it defaults to joblib

In [ ]:
mr_train_dataset = mr_train_dataset.map(resample, num_proc=1)
mr_test_dataset = mr_test_dataset.map(resample, num_proc=1)

Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. 

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(mr_train_dataset)-1)

ipd.Audio(data=np.asarray(mr_train_dataset[rand_int]["speech"]), autoplay=True, rate=16000)

It can be heard, that the speakers are all females and they change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
rand_int = random.randint(0, len(mr_train_dataset)-1)

print("Target text:", mr_train_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(mr_train_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", mr_train_dataset[rand_int]["sampling_rate"])

Target text: एक बस चार तासांपूर्वी येऊन गेली आणि पुढची बस यायला अजून फक्त साठ मिनिटं आहेत 
Input array shape: (124831,)
Sampling rate: 16000


The data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will again make use of the `map(...)` function.

First, we check that the data samples have the same sampling rate of 16kHz.
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* [Log-Mel features](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum). 
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
mr_train_dataset = mr_train_dataset.map(prepare_dataset, remove_columns=mr_train_dataset.column_names, batch_size=8, num_proc=4, batched=True)
mr_test_dataset = mr_test_dataset.map(prepare_dataset, remove_columns=mr_test_dataset.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
mr_train_dataset.description

'\n\n\n\n\n\n'

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Define checkpoints at steps to resume training if it fails

In [ ]:
import os
from transformers.trainer_utils import get_last_checkpoint
save_dir = "/content/gdrive/MyDrive/wav2vec2-large-xlsr-mr-marathi-2"
!ls {save_dir}
last_checkpoint = None

if os.path.exists(save_dir):
    last_checkpoint = get_last_checkpoint(save_dir)
    
print(last_checkpoint if last_checkpoint else 0)

ls: cannot access '/content/gdrive/MyDrive/wav2vec2-large-xlsr-mr-marathi-2': No such file or directory
0


Now, we can load the pretrained `XLSR-Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small (~2h of training data) and because Open SLR is quite noisy, fine-tuning Facebook's [wav2vec2-large-xlsr-53 checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough. 


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53" if not last_checkpoint else last_checkpoint, 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

Init Weights and Biases for auto logging and model tracking 

In [ ]:
%%capture
!pip install wandb -qqq
import wandb

In [ ]:
!wandb login

wandb: Currently logged in as: sumedhkhodke (use `wandb login --relogin` to force relogin)


In [ ]:
# W&B company account
%env WANDB_ENTITY = wandb
entity = os.environ["WANDB_ENTITY"]

# Choose the public W&B project
%env WANDB_PROJECT = xlsr
project_name = os.environ["WANDB_PROJECT"]

# Log your trained model to W&B as an Artifact
%env WANDB_LOG_MODEL = true 

# # Disable logging of gradients to speed things up a little
# %env WANDB_WATCH = false   

env: WANDB_ENTITY=wandb
env: WANDB_PROJECT=xlsr
env: WANDB_LOG_MODEL=true


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-mr-marathi-2",
#   output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=8, #reduced batch size from 16 to 8 to avoid Cuda OOM error on colab
  per_device_eval_batch_size=8, #reduced batch size from 16 to 8 to avoid Cuda OOM error on colab
  gradient_accumulation_steps=4, #increasd grad_accumu_steps from 2 to 4 to compensate for batch size...someone on the forum said xlsr likes total batch size of atleast 32
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True, #enable half precision fp for faster training
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=0.00045,
  warmup_steps=140, #10% of training data
  save_total_limit=2,
  # WANDB LOGGING: 
    report_to = 'wandb',  # enable logging to W&B
    run_name = 'mr-xlsr-3-35e',   # Name your run, optional
    load_best_model_at_end = True,  # This will ensure your best model will be uploaded to W&B
    metric_for_best_model='wer',    # Load best model based on "wer", not eval loss
    greater_is_better=False,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=mr_train_dataset,
    eval_dataset=mr_test_dataset,
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 180 and 240 minutes depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Open SLR 64*'s test data of Marathi, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how XLSR-Wav2Vec2's [checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) can be fine-tuned on a low-resource ASR dataset.


Clear the cached memory in GPU before starting the training

In [ ]:
torch.cuda.memory_summary(device=None)#, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1203 MB |    1203 MB |    1203 MB |       0 B  |\n|       from large pool |    1202 MB |    1202 MB |    1202 MB |       0 B  |\n|       from small pool |       1 MB |       1 MB |       1 MB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    1203 MB |    1203 MB |    1203 MB |       0 B  |\n|       from large pool |    1202 MB |    1202 MB |

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Sun Mar 28 17:28:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    34W / 250W |   2095MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# trainer.train()
# wandb.finish()
if last_checkpoint:
    print(f"last_checkpoint: {last_checkpoint}")
    train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
    wandb.finish()    # This will finish your WANDB run  
else:
    train_result = trainer.train()
    wandb.finish()    # This will finish your WANDB run  

wandb: Currently logged in as: sumedhkhodke (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
200,5.658800,3.355526,1.000000,30.133300,5.210000
400,2.658600,0.548290,0.594565,30.581600,5.134000
600,0.363300,0.193027,0.271769,30.930200,5.076000
800,0.156100,0.175891,0.232945,30.872100,5.086000
1000,0.102000,0.162028,0.215752,31.150900,5.040000
1200,0.079100,0.165674,0.197449,31.364000,5.006000


_runtime,12007
_timestamp,1616964618
_step,1320
train/loss,0.0791
train/learning_rate,5e-05
train/epoch,29.99
eval/loss,0.16567
eval/wer,0.19745
eval/runtime,31.364
eval/samples_per_second,5.006
train/train_runtime,12012.2238


_runtime,▁▂▃▅▆▇██
_timestamp,▁▂▃▅▆▇██
_step,▁▂▃▄▅▆▇█
train/loss,█▄▁▁▁▁
train/learning_rate,█▇▅▄▂▁
train/epoch,▁▂▄▅▆▇█
eval/loss,█▂▁▁▁▁
eval/wer,█▄▂▁▁▁
eval/runtime,▁▄▆▅▇█
eval/samples_per_second,█▅▃▄▂▁
train/train_runtime,▁




The resulting model of this notebook has been saved to [`sumedh/wav2vec2-large-xlsr-marathi`](https://huggingface.co/sumedh/wav2vec2-large-xlsr-marathi)

As a final check, let's load the model and verify that it indeed has learned to transcribe Marathi speech.

Let's first load the pretrained checkpoint. And we will just take the first example of the test set, run it through the model and take the `argmax(...)` of the logits to retrieve the predicted token ids.

In [ ]:
mr_test_dataset.column_names

['input_values', 'labels']

We adapted `mr_test_dataset` quite a bit so that the dataset instance does not contain the original sentence label anymore. Thus, we re-use the original dataset `all_data` to get the label of the first example.

##Usage

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

#test_dataset = load_dataset("common_voice", "{lang_id}", split="test[:2%]") #TODO: replace {lang_id} in your language code here. Make sure the code is one of the *ISO codes* of [this](https://huggingface.co/languages) site.
mr_test_dataset_new = all_data['test']

processor = Wav2Vec2Processor.from_pretrained("sumedh/wav2vec2-large-xlsr-marathi") 
model = Wav2Vec2ForCTC.from_pretrained("sumedh/wav2vec2-large-xlsr-marathi") 

resampler = torchaudio.transforms.Resample(48_000, 16_000) #first arg - input sample, second arg - output sample

# Preprocessing the datasets. We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
	speech_array, sampling_rate = torchaudio.load(batch["path_in_folder"])
	batch["speech"] = resampler(speech_array).squeeze().numpy()
	return batch

mr_test_dataset_new = mr_test_dataset_new.map(speech_file_to_array_fn)
inputs = processor(mr_test_dataset_new["speech"][:5], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
	logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", mr_test_dataset_new["actual_text"][:5])

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.



Prediction: ['अभिजीतने गेल्या महिन्यात हे शिखर सर केले आणि तो या शिखरावर आपले पाय रोवणारा अठरावा गिर्यारोहक बनला', 'एक बस चार तासांपूर्वी येऊन गेली आणि पुढची बस यायला अजून फक्त चाळीस मिनिटं आहेत', 'आज सहिंगापूरमध्ये किमान तापमान अठ्ठावीस अंश सेल्सिअस असेल असा अंदाज आहे', 'आता अ\u200dॅक्सिस बँकेची पाचवी शाखा फॅशन स्ट्रीटजवळ उघडते आहे परवा तिचा उद्घाटन समारंभ आहे', 'कुंभाराचेच नवे तर प्रत्येकाचे का महत्त्वाचे आहे असे मला वाटते']
Reference: ['अभिजीतने गेल्या महिन्यात हे शिखर सर केले आणि तो या शिखरावर आपले पाय रोवणारा अठरावा गिर्यारोहक बनला', 'एक बस चार तासांपूर्वी येऊन गेली आणि पुढची बस यायला अजून फक्त चाळीस मिनिटं आहेत', 'आज सिंगापूरमधे किमान तापमान अठ्ठावीस अंश सेल्सिअस असेल असा अंदाज आहे', 'आता अ\u200dॅक्सिस बँकेची पाचवी शाखा फॅशन स्ट्रीटजवळ उघडते आहे परवा तिचा उद्घाटन समारंभ आहे', 'कुंभाराचेच नव्हे तर प्रत्येकाचे काम महत्त्वाचे आहे असे मला वाटते']


## Evaluation

In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

# test_dataset = load_dataset("common_voice", "{lang_id}", split="test") #TODO: replace {lang_id} in your language code here. Make sure the code is one of the *ISO codes* of [this](https://huggingface.co/languages) site.
mr_test_dataset_new = all_data['test']
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("sumedh/wav2vec2-large-xlsr-marathi")
model = Wav2Vec2ForCTC.from_pretrained("sumedh/wav2vec2-large-xlsr-marathi") 
model.to("cuda")

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]' 
resampler = torchaudio.transforms.Resample(48_000, 16_000) #first arg - input sample, second arg - output sample

# Preprocessing the datasets. We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
	batch["actual_text"] = re.sub(chars_to_ignore_regex, '', batch["actual_text"]).lower()
	speech_array, sampling_rate = torchaudio.load(batch["path_in_folder"])
	batch["speech"] = resampler(speech_array).squeeze().numpy()
	return batch

mr_test_dataset_new = mr_test_dataset_new.map(speech_file_to_array_fn)

def evaluate(batch):
	inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

	with torch.no_grad():
		logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
		pred_ids = torch.argmax(logits, dim=-1)
		batch["pred_strings"] = processor.batch_decode(pred_ids)
	return batch

result = mr_test_dataset_new.map(evaluate, batched=True, batch_size=8)

print("WER: {}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["actual_text"])))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.



WER: 12.701053799223516


We have achieved a WER (Word Error Rate) of 12.70% 🤗 <br>
Not bad, right? 🤗